In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
tf.config.list_physical_devices()
# tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_path = '/kaggle/input/fer2013/train'
test_path = '/kaggle/input/fer2013/test'

In [5]:
img_height, img_width = 48, 48
batch_size = 128

In [6]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    train_path,
    color_mode='grayscale',
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 28709 files belonging to 7 classes.


In [7]:
test_ds = keras.preprocessing.image_dataset_from_directory(
    test_path,
    color_mode='grayscale',
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 7178 files belonging to 7 classes.


In [8]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(128, 48, 48, 1)
(128, 7)


In [9]:
class_names = train_ds.class_names
print(class_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [10]:
test_ds.class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [11]:
normalize = Rescaling(1./255)

In [12]:
train_ds = train_ds.map(lambda x, y: (normalize(x), y))
test_ds = test_ds.map(lambda x, y: (normalize(x), y))

In [13]:
num_classes = len(class_names)

In [14]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(48,48,1)),
    Conv2D(32, (3,3), padding='same',  activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(128,(3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Dropout(0.3),

    Conv2D(256,(5,5), padding='same', activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(512,(3,3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.25),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(128, activation='relu'),
    Dropout(0.25),
    
    Dense(num_classes, activation='softmax')
])

model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0

In [16]:
stop_early = EarlyStopping(monitor='val_accuracy', patience=20)   # Прекратить обучение, если нет прогресса
checkpoint = ModelCheckpoint('./checkpoints/best', monitor='val_accuracy',
                             save_best_only=True, save_weights_only=True)   # Сохранить лучшую эпоху

In [17]:
with tf.device('/GPU:0'):
    model.fit(
        train_ds.cache(),   #   Кэшируем датасеты целиком в память, потому что можем себе это позволить
        validation_data=test_ds.cache(),
        epochs=100,
        callbacks=[stop_early, checkpoint]
    )

Epoch 1/100


2023-06-16 16:22:28.652336: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


225/225 [==============================] - 78s 288ms/step - loss: 1.8829 - accuracy: 0.2318 - val_loss: 2.2250 - val_accuracy: 0.2471
Epoch 2/100
225/225 [==============================] - 9s 38ms/step - loss: 1.7095 - accuracy: 0.3018 - val_loss: 1.7858 - val_accuracy: 0.3157
Epoch 3/100
225/225 [==============================] - 9s 38ms/step - loss: 1.5152 - accuracy: 0.4053 - val_loss: 1.4911 - val_accuracy: 0.4207
Epoch 4/100
225/225 [==============================] - 8s 38ms/step - loss: 1.3712 - accuracy: 0.4704 - val_loss: 1.3376 - val_accuracy: 0.4900
Epoch 5/100
225/225 [==============================] - 8s 36ms/step - loss: 1.2841 - accuracy: 0.5137 - val_loss: 1.4383 - val_accuracy: 0.4893
Epoch 6/100
225/225 [==============================] - 9s 39ms/step - loss: 1.2150 - accuracy: 0.5382 - val_loss: 1.3505 - val_accuracy: 0.4901
Epoch 7/100
225/225 [==============================] - 8s 38ms/step - loss: 1.1584 - accuracy: 0.5676 - val_loss: 1.1521 - val_accuracy: 0.5635
Ep

In [22]:
model.load_weights('./checkpoints/best')

In [23]:
model.evaluate(test_ds.cache())

57/57 [==============================] - 3s 45ms/step - loss: 1.7850 - accuracy: 0.6413


[1.7850433588027954, 0.6412649750709534]

64% точность - вполне неплохо

In [21]:
model.save('model.h5')